In [1]:
from web3 import Web3
from datetime import datetime
import pandas as pd
import requests
import json
from IPython.display import Image

In [2]:
infura_url= 'https://mainnet.infura.io/v3/[YOUR KEY]'
web3= Web3(Web3.HTTPProvider (infura_url))
web3.isConnected()

True

In [24]:
sc_address="0xf074540eb83c86211F305E145eB31743E228E57d"
with open ("./bridge_abi.json") as f:
    abi= json.load (f)

In [25]:
bridgeContract = web3.eth.contract(address=sc_address, abi=abi)
bridgeContract.all_functions()

[<Function CHALLENGE_AMOUNT_DIVISOR()>,
 <Function TIME_SLOT_SIZE()>,
 <Function addBonder(address)>,
 <Function bondTransferRoot(bytes32,uint256,uint256)>,
 <Function bondWithdrawal(address,uint256,bytes32,uint256)>,
 <Function chainBalance(uint256)>,
 <Function challengePeriod()>,
 <Function challengeResolutionPeriod()>,
 <Function challengeTransferBond(bytes32,uint256)>,
 <Function confirmTransferRoot(uint256,bytes32,uint256,uint256,uint256)>,
 <Function crossDomainMessengerWrappers(uint256)>,
 <Function getBondForTransferAmount(uint256)>,
 <Function getBondedWithdrawalAmount(address,bytes32)>,
 <Function getChainId()>,
 <Function getChallengeAmountForTransferAmount(uint256)>,
 <Function getCredit(address)>,
 <Function getDebitAndAdditionalDebit(address)>,
 <Function getIsBonder(address)>,
 <Function getRawDebit(address)>,
 <Function getTimeSlot(uint256)>,
 <Function getTransferId(uint256,address,uint256,bytes32,uint256,uint256,uint256)>,
 <Function getTransferRoot(bytes32,uint256)>

In [26]:
#l1CanonicalToken
bridgeContract.functions.l1CanonicalToken().call()

'0xB0c7a3Ba49C7a6EaBa6cD4a96C55a1391070Ac9A'

In [27]:
##List of hop L1 bridge 
## source: https://github.com/hop-protocol/hop/blob/develop/packages/core/src/addresses/mainnet.ts 

bridges=['0x3666f603Cc164936C1b87e207F36BEBa4AC5f18a', '0x3E4a3a4796d16c0Cd582C382691998f7c06420B6', '0x22B1Cbb8D98a01a3B71D034BB899775A76Eb1cc2',
        '0x3d4Cc8A61c7528Fd86C55cfe061a78dCBA48EDd1',  '0x914f986a44AcB623A277d6Bd17368171FCbe4273', '0x893246FACF345c99e4235E5A7bbEE7404c988b96',  '0x36443fC70E073fe9D50425f82a3eE19feF697d62', '0x87269B23e73305117D0404557bAdc459CEd0dbEc',
          '0xf074540eb83c86211F305E145eB31743E228E57d'
        ]
#l1CanonicalToken_list=[]
tvl={}
## We take out the EtH because that one stores ETH not tokens
for bridge in bridges:
    with open ("./bridge_abi.json") as f:
        bridge_abi= json.load (f)

    bridgeContract = web3.eth.contract(address=bridge, abi=bridge_abi)    
    token_locked=bridgeContract.functions.l1CanonicalToken().call()
    
    with open ("./erc20_abi.json") as f:
        erc_abi= json.load (f)
    collateralcontract = web3.eth.contract(address=token_locked, abi=erc_abi)
    decimal=collateralcontract.functions.decimals().call()
    locked= (collateralcontract.functions.balanceOf(bridge).call())*10**-(int(decimal))
    print (collateralcontract.functions.name().call(), locked)
    tvl[collateralcontract.functions.name().call()]=locked

USD Coin 9033573.495695999
Tether USD 2039626.6284339998
Matic Token 1125462.7843598279
Dai Stablecoin 3033540.347161223
Hop 25053569.15818467
Synthetix Network Token 591010.9652927803
Synth sUSD 1221293.989997617
Rocket Pool ETH 1153.5951250765336
MAGIC 1250081.3799648664


In [ ]:
#l1CanonicalToken0='0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48' ###USD
#l1CanonicalToken0='0xdAC17F958D2ee523a2206206994597C13D831ec7' ###USDT
#l1CanonicalToken0='0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0' ###Matic
#l1CanonicalToken0='0x6B175474E89094C44Da98b954EedeAC495271d0F' ###DAI
#l1CanonicalToken0='0xc5102fE9359FD9a28f877a67E36B0F050d81a3CC' ###Hop
#l1CanonicalToken0='0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F' ###SNX
#l1CanonicalToken0='0xae78736Cd615f374D3085123A210448E74Fc6393' ###rETH
#l1CanonicalToken0='0xB0c7a3Ba49C7a6EaBa6cD4a96C55a1391070Ac9A' ###Magic

In [28]:
tvl

{'USD Coin': 9033573.495695999,
 'Tether USD': 2039626.6284339998,
 'Matic Token': 1125462.7843598279,
 'Dai Stablecoin': 3033540.347161223,
 'Hop': 25053569.15818467,
 'Synthetix Network Token': 591010.9652927803,
 'Synth sUSD': 1221293.989997617,
 'Rocket Pool ETH': 1153.5951250765336,
 'MAGIC': 1250081.3799648664}

In [29]:
url = "https://api.coingecko.com/api/v3/simple/price?ids=ethereum&vs_currencies=usd"
eth_price = requests.get(url).json()["ethereum"]["usd"]

url = "https://api.coingecko.com/api/v3/simple/price?ids=matic-network&vs_currencies=usd"
matic_price = requests.get(url).json()["matic-network"]["usd"]

url = "https://api.coingecko.com/api/v3/simple/price?ids=usd-coin&vs_currencies=usd"
usdc_price = requests.get(url).json()["usd-coin"]["usd"]

url = "https://api.coingecko.com/api/v3/simple/price?ids=tether&vs_currencies=usd"
usdt_price = requests.get(url).json()["tether"]["usd"]

url = "https://api.coingecko.com/api/v3/simple/price?ids=dai&vs_currencies=usd"
dai_price = requests.get(url).json()["dai"]["usd"]

url = "https://api.coingecko.com/api/v3/simple/price?ids=hop-protocol&vs_currencies=usd"
hop_price = requests.get(url).json()["hop-protocol"]["usd"]

url = "https://api.coingecko.com/api/v3/simple/price?ids=havven&vs_currencies=usd"
snx_price = requests.get(url).json()["havven"]["usd"]

url = "https://api.coingecko.com/api/v3/simple/price?ids=nusd&vs_currencies=usd"
susd_price = requests.get(url).json()["nusd"]["usd"]

url = "https://api.coingecko.com/api/v3/simple/price?ids=rocket-pool-eth&vs_currencies=usd"
rETH_price = requests.get(url).json()["rocket-pool-eth"]["usd"]

url = "https://api.coingecko.com/api/v3/simple/price?ids=magic&vs_currencies=usd"
magic_price = requests.get(url).json()["magic"]["usd"]

In [38]:
prices_dic={'USD Coin': usdc_price,
 'Tether USD': usdt_price,
 'Matic Token': matic_price,
 'Dai Stablecoin': dai_price,
 'Hop': hop_price,
 'Synthetix Network Token': snx_price,
 'Synth sUSD': susd_price,
 'Rocket Pool ETH': rETH_price,
 'MAGIC': magic_price,
 'ethereum': eth_price}

In [39]:
prices_dic

{'USD Coin': 0.999487,
 'Tether USD': 0.999825,
 'Matic Token': 0.721981,
 'Dai Stablecoin': 0.999436,
 'Hop': 0.063698,
 'Synthetix Network Token': 2.82,
 'Synth sUSD': 0.996588,
 'Rocket Pool ETH': 1986.29,
 'MAGIC': 0.726983,
 'ethereum': 1843.76}

In [37]:
ethereum= web3.eth.get_balance('0xb8901acB165ed027E32754E0FFe830802919727f')*10**-18

In [42]:
df=pd.DataFrame.from_dict(tvl, orient='index', columns=['holding_amount'])
df.head()

,holding_amount
USD Coin,9.033573e+06
Tether USD,2.039627e+06
Matic Token,1.125463e+06
Dai Stablecoin,3.033540e+06
Hop,2.505357e+07


In [47]:
df['price'] = df.index.map(prices_dic)
df.reset_index(inplace=True)
df.head()

,index,holding_amount,price
0,USD Coin,9.033573e+06,0.999487
1,Tether USD,2.039627e+06,0.999825
2,Matic Token,1.125463e+06,0.721981
3,Dai Stablecoin,3.033540e+06,0.999436
4,Hop,2.505357e+07,0.063698


In [48]:
eth_row = {'index':'ethereum', 'holding_amount': ethereum, 'price': eth_price}
df = df.append(eth_row, ignore_index=True)

# Add the new row using loc


In [49]:
df.head()

,index,holding_amount,price
0,USD Coin,9.033573e+06,0.999487
1,Tether USD,2.039627e+06,0.999825
2,Matic Token,1.125463e+06,0.721981
3,Dai Stablecoin,3.033540e+06,0.999436
4,Hop,2.505357e+07,0.063698


In [50]:
df['amount_usd']=df['holding_amount']*df['price']

In [51]:
tvl=df.amount_usd.sum()
tvl

45905515.596188046

In [52]:
tvl/1000000

45.905515596188046